# Mean-Field Equilibrium vs N-player Equilibrium in the SIR model with vaccinations

In [1]:
import numpy as np
import ternary
from matplotlib import pyplot, gridspec

System parameters, costs and initial conditions

In [2]:
# system parameters
gamma = 73.0     # infection rate
rho = 36.5    # recovery rate
vac_min = 0.0   # minimum vaccination rate
theta = 10.0  # maximum vaccination rate
T = 1           # time horizon

c_V = 0.5       # cost per unit time of vaccination
c_I = 36.5      # cost per unit time of infection

# initial conditions
S0 = 0.4        # proportion of susceptible at t=0
I0 = 0.4        # proportion of infected at t=0


The function to compute the mean-field equilibrium

In [3]:
def mf_equilibrium():
    
    thr_mass=0;
    thr_pl0=-1;
    best_pol=np.zeros(C+1);
    count=0
    
    while (thr_mass != thr_pl0) and (count<500):
        
        x_S = np.zeros(C+1); x_I = np.zeros(C+1);
        policy = np.zeros(C+1);
        x_S[0]=S0; x_I[0]=I0;
        for i in range(C):
            if i<thr_mass:
                policy[i]=theta;
            else:
                policy[i]=vac_min;
            x_S[i+1] = x_S[i]+(-gamma*x_I[i]*x_S[i] - policy[i]*x_S[i])/C;
            x_I[i+1] = x_I[i]+(gamma*x_I[i]*x_S[i] - rho*x_I[i])/C;    

        for count in range(C):
            t_ = C - count ;
            if c_V - vs[t_] < 0:
                best_pol[t_-1]=theta;
                p_s_s=1 - 1.*gamma*x_I[t_-1]/C - 1.*theta/C;
                p_s_i=1.*gamma*x_I[t_-1]/C; 
                vs[t_-1]= theta * c_V /C + p_s_s * vs[t_] + p_s_i * vi[t_];
            else:
                best_pol[t_-1]=vac_min;
                p_s_s=1 - 1.*gamma*x_I[t_-1]/C - vac_min/C;
                p_s_i=gamma*x_I[t_-1]/C; 
                vs[t_-1]= vac_min * c_V/C + p_s_s * vs[t_] + p_s_i * vi[t_];   
            p_i_i=1-rho/C; 
            vi[t_-1] = c_I/C + p_i_i * vi[t_];   
         
        thr=0;
        for i in range(C+1):
            if vs[i]<c_V:
                thr_pl0=i;
                break;
                
        count+=1
        
        if thr_mass > thr_pl0:
            thr_mass = thr_mass - 1;
        if thr_mass < thr_pl0:
            thr_mass = thr_pl0 + 1;  
    
    return thr_pl0, vs, vi;

We compute the mean-field equilibrium when the initial state is (0.4,0.4):

In [4]:
C=10000;
x_S = np.zeros(C+1); x_I = np.zeros(C+1)
vs = np.zeros(C+1); vi = np.zeros(C+1)
t_critical, vs, vi = mf_equilibrium();
print("Cost of the mean-field equilibrium when the initial state is ", S0,I0)
cost_mfe=vs[0]*S0+vi[0]*I0
print(cost_mfe)

Cost of the mean-field equilibrium when the initial state is  0.4 0.4
0.6907513845221682


Functions to compute the equilibrium of the N players game

In [5]:
def trans_prob(m_s, m_i, m_pi, pl0_pi, N, C):
    p_x = np.zeros(3)
    p_m = np.zeros(3)
    # transition probabilities of Player 0
    p_x[0] = 1. * gamma / C * m_i / N
    p_x[1] = 1. * rho / C
    p_x[2] = 1. * pl0_pi / C
    # transition probabilities of the mass
    p_m[0] = 1. * gamma * N / C * m_s / N * m_i / N
    p_m[1] = 1. * m_i / N * rho * N / C
    p_m[2] = 1. * m_s / N * m_pi * N / C

    return p_x, p_m


def best_response_n_players(m_pol):

    if C < N * max(theta, gamma, rho): raise Exception('Higher value of C required')

    # checking that S0*N and I0*N are integers
    if not (1. * S0 * N).is_integer():
        raise Exception('S0*N is not integer')

    if not (1. * I0 * N).is_integer():
        raise Exception('I0*N is not integer')

    # Pl0 cost at susceptible and infected state: Js and Ji
    Js = np.zeros((C + 1, N + 1, N + 1))
    Ji = np.zeros((C + 1, N + 1, N + 1))
    # Best response of Player 0
    BR_pol = np.zeros((C + 1, N + 1, N + 1))

    for t in range(C, 0, -1):
        for ms in range(0, N + 1):
            for mi in range(0, N - ms + 1):
                if c_V - Js[t, ms, mi] < 0:
                    BR_pol[t - 1, ms, mi] = theta
                else:
                    BR_pol[t - 1, ms, mi] = 0
                px, pm = trans_prob(ms, mi, m_pol[t - 1, ms, mi], BR_pol[t - 1, ms, mi], N, C)
                Js[t - 1, ms, mi] = (
                    c_V * BR_pol[t - 1, ms, mi] / C +                               # pl0 instantaneous cost at susceptible state
                    (px[0] * Ji[t, ms, mi] if mi > 0 else 0) +                      # pl0 infection
                    (px[2] * 0) +                                                   # pl0 vaccination: the cost when it is recovered is 0
                    (pm[0] * Js[t, ms - 1, mi + 1] if ms > 0 else 0) +              # infection of mass
                    (pm[1] * Js[t, ms, mi - 1] if mi > 0 else 0) +                  # recovery of mass
                    (pm[2] * Js[t, ms - 1, mi] if ms > 0 else 0) +                  # vaccination of mass
                    (1 - px[0] - px[2] - pm[0] - pm[1] - pm[2]) * Js[t, ms, mi])    # no transition
                Ji[t - 1, ms, mi] = (
                    c_I / C +                                               # pl0 instantaneous cost at infected state
                    (px[1] * 0) +                                           # pl0 recovery: the cost when it is recovered is 0
                    (pm[0] * Ji[t, ms - 1, mi + 1] if ms > 0 else 0) +      # infection of mass
                    (pm[1] * Ji[t, ms, mi - 1] if mi > 0 else 0) +          # recovery of mass
                    (pm[2] * Ji[t, ms - 1, mi] if ms > 0 else 0) +          # vaccination of mass
                    (1 - px[1] - pm[0] - pm[1] - pm[2]) * Ji[t, ms, mi])    # no transition
    return BR_pol, Js, Ji


def equilibrium_n_players():
    # the policy of the mass is never vaccinate
    m_pol = np.zeros((C + 1, N + 1, N + 1))
    # the policy of the mass is to always vaccinate with max rate
    # m_pol=theta*np.ones((C+1,N+1,N+1));

    BR_pol, Js, Ji = best_response_n_players(m_pol)
    # print(BR_pol[0])

    while 0 < 1:
        if np.allclose(BR_pol, m_pol):
            print("Equilibrium found. Equilibrium policy at t=0:")
            print(BR_pol[0])
            break
        else:
            print("BR_pol and m_pol do not coindice. Updating m_pol...")
            #print("The optimal cost in S state when S0 and I0 are 0.4:")
            #print(Js[0, int(S0 * N), int(I0 * N)])
            #print(np.nonzero(BR_pol-m_pol))
            # print(BR_pol[0])
            m_pol = BR_pol
            BR_pol, Js, Ji = best_response_n_players(m_pol)
    
    return BR_pol, Js, Ji

We compute the N-player equilibrium when N=5:

In [6]:
N = 5            # number of players
C = T * N * 100  # number of the intervals of the time grid
eq_pol = np.zeros((C + 1, N + 1, N + 1))
cost_s = np.zeros((C + 1, N + 1, N + 1))
cost_i = np.zeros((C + 1, N + 1, N + 1))
eq_pol_n5, cost_s_n5, cost_i_n5 = equilibrium_n_players()
print("Cost of the 5-player equilibrium when the initial state is ",S0,I0)
print(cost_s_n5[0,int(S0*N),int(I0*N)]*S0+cost_i_n5[0,int(S0*N),int(I0*N)]*I0)

BR_pol and m_pol do not coindice. Updating m_pol...
Equilibrium found. Equilibrium policy at t=0:
[[ 0.  0.  0. 10. 10. 10.]
 [ 0.  0. 10. 10. 10.  0.]
 [ 0.  0. 10. 10.  0.  0.]
 [ 0.  0. 10.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
Cost of the 5-player equilibrium when the initial state is  0.4 0.4
0.6345379909054636


We compute the N-player equilibrium when N=10:

In [ ]:
N = 10           # number of players
C = T * N * 100  # number of the intervals of the time grid
eq_pol = np.zeros((C + 1, N + 1, N + 1))
cost_s = np.zeros((C + 1, N + 1, N + 1))
cost_i = np.zeros((C + 1, N + 1, N + 1))
eq_pol_n10, cost_s_n10, cost_i_n10 = equilibrium_n_players()
print("Cost of the 10-players equilibrium when the initial state is ",S0,I0)
cost_eq_n10=cost_s_n10[0,int(S0*N),int(I0*N)]*S0+cost_i_n10[0,int(S0*N),int(I0*N)]*I0
print(cost_eq_n10)

BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m

We compute the N-player equilibrium when N=20:

In [ ]:
N = 20            # number of players
C = T * N * 100  # number of the intervals of the time grid
eq_pol = np.zeros((C + 1, N + 1, N + 1))
cost_s = np.zeros((C + 1, N + 1, N + 1))
cost_i = np.zeros((C + 1, N + 1, N + 1))
eq_pol_n20, cost_s_n20, cost_i_n20 = equilibrium_n_players()
print("Cost of the 20-players equilibrium when the initial state is ",S0,I0)
cost_eq_n20=cost_s_n20[0,int(S0*N),int(I0*N)]*S0+cost_i_n20[0,int(S0*N),int(I0*N)]*I0;
print(cost_eq_n20)

BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m_pol do not coindice. Updating m_pol...
BR_pol and m

We compute the N-player equilibrium when N=30:

In [ ]:
N = 30            # number of players
C = T * N * 100  # number of the intervals of the time grid
eq_pol = np.zeros((C + 1, N + 1, N + 1))
cost_s = np.zeros((C + 1, N + 1, N + 1))
cost_i = np.zeros((C + 1, N + 1, N + 1))
eq_pol_n30, cost_s_n30, cost_i_n30 = equilibrium_n_players()
print("Cost of the 30-players equilibrium when the initial state is ",S0,I0)
cost_eq_n30=cost_s_n30[0,int(S0*N),int(I0*N)]*S0+cost_i_n30[0,int(S0*N),int(I0*N)]*I0
print(cost_eq_n30)

The average cost of the Nash Equilibrium when N=30 and the initial state is (0.4,0.4) is 0.6737.

We compute the N-player equilibrium when N=40:

In [ ]:
N = 40            # number of players
C = T * N * 100  # number of the intervals of the time grid
eq_pol = np.zeros((C + 1, N + 1, N + 1))
cost_s = np.zeros((C + 1, N + 1, N + 1))
cost_i = np.zeros((C + 1, N + 1, N + 1))
eq_pol_n40, cost_s_n40, cost_i_n40 = equilibrium_n_players()
print("Cost of the 40-players equilibrium when the initial state is ",S0,I0)
cost_eq_n40=cost_s_n40[0,int(S0*N),int(I0*N)]*S0+cost_i_n40[0,int(S0*N),int(I0*N)]*I0
print(cost_eq_n40)

The average cost of the Nash Equilibrium when N=40 and the initial state is (0.4,0.4) is 0.6759.

We compute the N-player equilibrium when N=50:

In [ ]:
N = 50            # number of players
C = T * N * 100  # number of the intervals of the time grid
eq_pol = np.zeros((C + 1, N + 1, N + 1))
cost_s = np.zeros((C + 1, N + 1, N + 1))
cost_i = np.zeros((C + 1, N + 1, N + 1))
eq_pol_n50, cost_s_n50, cost_i_n50 = equilibrium_n_players()
print("Cost of the 50-players equilibrium when the initial state is ",S0,I0)
cost_eq_n50=cost_s_n50[0,int(S0*N),int(I0*N)]*S0+cost_i_n50[0,int(S0*N),int(I0*N)]*I0
print(cost_eq_n50)

The average cost of the Nash Equilibrium when N=50 and the initial state is (0.4,0.4) is 0.6772.

# Plot of the MFE and of N-player equilibrium (N=5,10,20,30,40,50)

Note: the value b=0.26 corresponds to the best fit of the type a+b/N to the mean field equibrium for the following parameters: gamma = 73.0, rho = 36.5, vac_min = 0.0, theta = 10.0, T = 1, c_V = 0.5, c_I = 36.5, S0 = 0.4, I0 = 0.4. For other parameters, the value of b must be recalculated

In [ ]:
b=0.26 

import matplotlib.pyplot as plt
%matplotlib inline
N = np.array([5, 10, 20, 30, 40, 50])
cost = np.array([cost_eq_n5, cost_eq_n10, cost_eq_n20, cost_eq_n30, cost_eq_n40, cost_eq_n50])
mfg = cost_mfe
plt.plot(N,cost)
plt.plot(N,np.ones(6)*cost_mfe,'--')
   
plt.plot(N,cost_mfe-b/N,'-.')
plt.legend(['cost of equilibrium for $N$ players','mf cost','0.6824-0.26/N'])
plt.xlabel('N')